<a href="https://colab.research.google.com/github/bechirzammouri/translation-task-hf/blob/main/clm_task_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00


In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `rw_hf_token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `rw_hf_

In [ ]:
import pandas as pd

books = pd.read_parquet("hf://datasets/Helsinki-NLP/opus_books/en-fr/train-00000-of-00001.parquet")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


In [ ]:
from datasets import Dataset
books = Dataset.from_pandas(books)

In [ ]:

books

Dataset({
    features: ['id', 'translation'],
    num_rows: 127085
})

In [ ]:
print(books.shape)

(127085, 2)


In [ ]:
print(books.shape[0])

127085


In [ ]:
books = books.train_test_split(test_size=0.2)

In [ ]:
books["train"][:5]

{'id': ['58848', '45256', '49845', '64403', '85703'],
 'translation': [{'en': 'But the witchcraft of the goat with the golden hoofs was a very innocent species of magic.',
   'fr': 'Pourtant les sorcelleries de la chèvre aux pattes dorées étaient de bien innocentes malices.'},
  {'en': '"I mean that miserable d’Artagnan."',
   'fr': "-- Je veux parler de ce misérable d'Artagnan."},
  {'en': 'He went to the small parlour, but the three millers had to be got out first, and during the whole time necessary for laying the cloth, Binet remained silent in his place near the stove. Then he shut the door and took off his cap in his usual way.',
   'fr': 'Il se dirigea vers la petite salle; mais il fallut d’abord en faire sortir les trois meuniers; et, pendant tout le temps que l’on fut à mettre son couvert, Binet resta silencieux à sa place, auprès du poêle; puis il ferma la porte et retira sa casquette, comme d’usage.'},
  {'en': 'It oozed.', 'fr': 'Ça suintait !'},
  {'en': 'Madame Derville h

In [ ]:
from transformers import AutoTokenizer
#loading a tokenizer model T5
checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
source_lang = "en"
target_lang = "fr"
prefix = "translate English to French: "


def preprocess_function(examples):
    inputs = prefix + examples["translation"][source_lang]
    targets =  examples["translation"][target_lang]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

In [ ]:
tokenized_books = books.map(preprocess_function)


Map:   0%|          | 0/101668 [00:00<?, ? examples/s]

Map:   0%|          | 0/25417 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## defining evaluation metric and preparing the evaluation

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np

def postprocess_text(preds, labels):
  preds = [ pred.strip() for pred in preds]
  labels = [ [ label.strip()] for label in labels]

  return preds , labels

def compute_metrics(eval_preds):
  preds , labels = eval_preds
  if isinstance(preds, tuple):
    preds = preds[0]

  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens = True)

  labels = np.where(labels != -100 , labels , tokenizer.pad_token_id )
  decoded_labels = tokenizer.batch_decode(labels,skip_special_token = True)

  decoded_preds , decoded_labels = postprocess_text(decoded_preds , decoded_labels)

  result = metric.compute(predictions=decoded_preds, references=decoded_labels)
  result = {"bleu" : result ["score"]}

  prediction_lens = [np.count_nonzero(pred!= tokenizer.pad_token_id) for pred in preds]
  result["gen_len"] = np.mean(prediction_lens)
  result = { k: round(v,4) for k,v in result.items()}

  return result



*   Training T5 model



In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = "bechir_opus_books_model",
    eval_strategy="epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16 ,
    per_device_eval_batch_size= 16 ,
    weight_decay=0.01 ,
    save_total_limit=3 ,
    num_train_epochs = 2,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_books["train"],
    eval_dataset=tokenized_books["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

/tmp/ipython-input-47-4077707952.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import AutoModelForSeq2SeqLM ,AutoTokenizer

text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

tokenizer = AutoTokenizer.from_pretrained("bechirzammouri/bechir_opus_books_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

model = AutoModelForSeq2SeqLM.from_pretrained("bechirzammouri/bechir_opus_books_model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

tokenizer.decode(outputs[0], skip_special_tokens=True)